In [1]:
import findspark
import plotly.express as px

findspark.init('/opt/spark')
import pyspark.sql.functions as f
import pyspark.sql.types as t
from pyspark.sql import SparkSession, Window

In [3]:
spark = SparkSession \
            .builder \
            .config("spark.sql.broadcastTimeout", "360000") \
            .config("spark.driver.memory", '13G') \
            .config("spark.executor.memory", '13G') \
            .config("spark.driver.maxResultSize", '4G')\
            .config("spark.sql.debug.maxToStringFields", 100000)\
            .getOrCreate() 
clicks_df = spark.read.parquet('Dados/metricas_parquet/*')

In [5]:
#clicks_df = spark.read.csv('Dados/*', header=True)
#clicks_df.coalesce(108).write.parquet('Dados/metricas_parquet')

24/04/02 12:01:14 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
24/04/02 12:01:23 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers


In [3]:
clicks_df.count()

33542342

In [4]:
clicks_df.select('productSKU', 'day', 'storeId').distinct().count()

33542342

## Tratamento de atributos

In [5]:
clicks_df.printSchema()

root
 |-- productSKU: string (nullable = true)
 |-- totalClicks: string (nullable = true)
 |-- totalImpressions: string (nullable = true)
 |-- day: string (nullable = true)
 |-- storeId: string (nullable = true)



In [4]:
clicks_df = clicks_df.withColumn("totalClicks", f.col("totalClicks").cast(t.IntegerType()))\
                        .withColumn("totalImpressions", f.col("totalImpressions").cast(t.IntegerType()))\
                        .withColumn("day", f.to_date(f.col("day"), "yyyyMMdd"))

## Criação de atributos

In [5]:
clicks_df = clicks_df.withColumn('atributtes', f.split(f.col('productSKU'), '-'))\
        .withColumn('brand', f.col('atributtes')[0])\
        .withColumn('model', f.col('atributtes')[1])\
        .withColumn('color', f.col('atributtes')[3])\
        .withColumn('size', f.col('atributtes')[4])\
        .drop('atributtes')

In [6]:
clicks_df = clicks_df.withColumn('convertion_perc', f.col('totalClicks')/f.col('totalImpressions')*100)

## Verificação de atributos

In [9]:
clicks_df.where('color is null').count()

33542341

In [10]:
clicks_df.where('size is null').count()

33542342

In [11]:
clicks_df[['totalClicks', 'totalImpressions', 'convertion_perc']].summary().show()

+-------+-----------------+------------------+------------------+
|summary|      totalClicks|  totalImpressions|   convertion_perc|
+-------+-----------------+------------------+------------------+
|  count|         33542342|          33542342|          33542342|
|   mean|403.0372951000261| 23776.58708375223|1.9819680732537768|
| stddev|6733.765690797448|329162.11916424433| 23.67754506648008|
|    min|                0|                 1|               0.0|
|    25%|                0|               649|               0.0|
|    50%|                0|              2338|               0.0|
|    75%|                0|              8238|               0.0|
|    max|          3559989|         166130472|           40000.0|
+-------+-----------------+------------------+------------------+



In [12]:
clicks_df.where('convertion_perc > 100').count()

17090

In [13]:
clicks_df.where('convertion_perc > 100').show()

+------------+-----------+----------------+----------+----------+-----+-----+-----+----+------------------+
|  productSKU|totalClicks|totalImpressions|       day|   storeId|brand|model|color|size|   convertion_perc|
+------------+-----------+----------------+----------+----------+-----+-----+-----+----+------------------+
|NR2-0245-006|        576|             288|2023-06-05|L_NETSHOES|  NR2| 0245| null|null|             200.0|
|D40-6542-014|        206|             103|2023-06-05|L_NETSHOES|  D40| 6542| null|null|             200.0|
|560-0075-058|        106|              56|2023-06-05|L_NETSHOES|  560| 0075| null|null|189.28571428571428|
|D45-0216-138|        138|              70|2023-06-05|L_NETSHOES|  D45| 0216| null|null|197.14285714285717|
|4W9-0132-138|        958|             441|2023-06-05|L_NETSHOES|  4W9| 0132| null|null|217.23356009070295|
|AX9-0159-203|        532|             266|2023-06-05|L_NETSHOES|  AX9| 0159| null|null|             200.0|
|D94-2871-006|        268|  

In [ ]:
#clicks_df.where('convertion_perc > 100').groupBy('brand').count().orderBy('count', ascending = False).show()

In [9]:
clicks_df = clicks_df.drop('size', 'color').where('convertion_perc <= 100')

### Marca

In [16]:
clicks_df.select('brand').distinct().count()

13060

In [18]:
clicks_df.groupBy('storeId').agg(f.countDistinct('brand').alias('brand_amount')).show()

+----------+------------+
|   storeId|brand_amount|
+----------+------------+
| L_ZATTINI|        9056|
|L_NETSHOES|       12607|
+----------+------------+



In [15]:
clicks_df.where('brand is null').count()

0

In [10]:
brand_agg = clicks_df.groupBy('brand').agg(f.count('*').alias('total_ads'),
                                           f.countDistinct('model').alias('model_amount'),
                                           f.avg('convertion_perc').alias('average_convertion'))

In [11]:
brand_agg_pd = brand_agg.toPandas()

In [12]:
brand_agg_pd.sort_values('total_ads', ascending = False).head(10)

,brand,total_ads,model_amount,average_convertion
7912,FB8,321845,5191,2.426451
12213,2I3,290288,4564,2.477614
6598,2I8,260656,4141,1.740164
746,2I2,222757,3552,2.419882
6322,39W,217172,4310,2.378403
2035,2DD,216566,3374,2.051734
4853,NFN,214723,3688,1.548115
6776,2FU,194300,3102,2.586628
10264,PFN,187955,3293,2.110434
8233,D84,182252,3006,2.160293


In [13]:
brand_agg_pd.sort_values('model_amount', ascending = False).head(10)

,brand,total_ads,model_amount,average_convertion
2493,2LY,88927,5541,0.631796
1683,BUE,121876,5219,0.763839
7912,FB8,321845,5191,2.426451
12213,2I3,290288,4564,2.477614
6322,39W,217172,4310,2.378403
6598,2I8,260656,4141,1.740164
4853,NFN,214723,3688,1.548115
1682,2IC,114715,3614,2.649178
10657,J4N,49123,3591,0.278866
746,2I2,222757,3552,2.419882


In [14]:
brand_agg_pd.sort_values('average_convertion', ascending = False).head(10)

,brand,total_ads,model_amount,average_convertion
12852,KEP,1,1,100.000000
1943,FJ7,1,1,100.000000
9949,K51,1,1,100.000000
12717,054,1,1,100.000000
9810,960,1,1,100.000000
4355,492,2,1,100.000000
1157,03S,1,1,100.000000
12458,6LX,1,1,100.000000
6140,0O1,3,1,95.161290
10643,732,3,1,66.666667


In [22]:
clicks_df.where(f.col('brand').isin(['054', 'FJ7', '492', '0O1', '732', 'KKT'])).show()

+----------+-----------+----------------+----------+----------+-----+-----+-----------------+
|productSKU|totalClicks|totalImpressions|       day|   storeId|brand|model|  convertion_perc|
+----------+-----------+----------------+----------+----------+-----+-----+-----------------+
|  FJ7-0000|         23|              23|2023-06-05|L_NETSHOES|  FJ7| 0000|            100.0|
|  732-0050|         24|              24|2023-06-14|L_NETSHOES|  732| 0050|            100.0|
|  0O1-0006|         23|              23|2023-06-04|L_NETSHOES|  0O1| 0006|            100.0|
|  KKT-0002|          0|               8|2023-06-19| L_ZATTINI|  KKT| 0002|              0.0|
|  492-9585|         32|              32|2023-06-21|L_NETSHOES|  492| 9585|            100.0|
|  732-0050|         33|              66|2023-06-21|L_NETSHOES|  732| 0050|             50.0|
|  054-4800|         20|              20|2023-06-27|L_NETSHOES|  054| 4800|            100.0|
|  492-9585|         16|              16|2023-06-03|L_NETSHO

In [15]:
brand_agg[['total_ads', 'model_amount', 'average_convertion']].summary().show()

+-------+------------------+------------------+------------------+
|summary|         total_ads|      model_amount|average_convertion|
+-------+------------------+------------------+------------------+
|  count|             13060|             13060|             13060|
|   mean|2567.0177641653904| 67.53445635528331|1.8159491986318714|
| stddev|10996.002976041167|239.47310252870236| 3.716527140568981|
|    min|                 1|                 0|               0.0|
|    25%|                64|                 3|0.6211180124223602|
|    50%|               285|                10|1.3554581441866942|
|    75%|              1297|                40|2.1678368038766873|
|    max|            321845|              5541|             100.0|
+-------+------------------+------------------+------------------+



In [9]:
brand_ad_amount = clicks_df.groupBy('brand').count().toPandas()

In [10]:
fig = px.box(brand_ad_amount, y="count")
fig.update_layout(width = 800, height = 500, paper_bgcolor = 'white', plot_bgcolor = 'white')
#fig.update_traces(boxpoints=False) 
fig.show()

In [17]:
clicks_df.where('brand == "PEP"').show()

+------------+-----------+----------------+----------+----------+-----+-----+------------------+
|  productSKU|totalClicks|totalImpressions|       day|   storeId|brand|model|   convertion_perc|
+------------+-----------+----------------+----------+----------+-----+-----+------------------+
|    PEP-0002|       1955|            5534|2023-06-05|L_NETSHOES|  PEP| 0002|35.327069027827974|
|PEP-0002-006|          0|             374|2023-06-06|L_NETSHOES|  PEP| 0002|               0.0|
|    PEP-0002|          0|            2186|2023-06-06|L_NETSHOES|  PEP| 0002|               0.0|
|    PEP-0002|       7911|          144745|2023-06-14|L_NETSHOES|  PEP| 0002| 5.465473764206018|
|PEP-0002-006|          0|            1254|2023-06-13|L_NETSHOES|  PEP| 0002|               0.0|
|    PEP-0002|       1278|           33380|2023-06-13|L_NETSHOES|  PEP| 0002|3.8286399041342123|
|PEP-0002-006|          0|              13|2023-06-07|L_NETSHOES|  PEP| 0002|               0.0|
|    PEP-0002|          0|    

In [16]:
brand_agg_pd.quantile([.1, .9], numeric_only=True)

,total_ads,model_amount,average_convertion
0.1,22.0,1.0,0.00000
0.9,4844.0,136.0,3.60309


In [16]:
fig = px.scatter(brand_agg_pd[(brand_agg_pd.total_ads >= 22) & (brand_agg_pd.total_ads <= 4844) &
                              (brand_agg_pd.model_amount >= 1) & (brand_agg_pd.model_amount <= 136)], 
                 x="total_ads", y="model_amount", 
                 size='average_convertion', hover_data=['brand'])
fig.update_layout(width = 800, height = 500, paper_bgcolor = 'white', plot_bgcolor = 'white')
fig.show()

In [18]:
brand_agg_pd[(brand_agg_pd.total_ads >= 22) & (brand_agg_pd.total_ads <= 4844) &
                              (brand_agg_pd.model_amount >= 1) & (brand_agg_pd.model_amount <= 136)]\
                              .sort_values('average_convertion', ascending = False).head(10)

,brand,total_ads,model_amount,average_convertion
6481,0VU,34,2,26.668860
6804,270,70,3,21.975700
3115,327,86,6,19.965831
7690,FPV,29,4,17.425287
1371,AWU,43,2,17.382018
7550,T64,62,3,16.477181
1228,8I6,30,1,16.351766
7512,3WY,60,1,13.134854
5053,5LY,26,1,12.756251
6135,F93,24,2,12.500000


In [19]:
brand_agg_pd.corr(numeric_only=True).T

,total_ads,model_amount,average_convertion
total_ads,1.000000,0.920809,-0.001570
model_amount,0.920809,1.000000,-0.014443
average_convertion,-0.001570,-0.014443,1.000000


In [32]:
clicks_df.groupBy('storeId').agg(f.avg('convertion_perc').alias('avg_convertion')).show()

+----------+------------------+
|   storeId|    avg_convertion|
+----------+------------------+
| L_ZATTINI|1.4969755054657108|
|L_NETSHOES|1.9560124738903302|
+----------+------------------+



## Dia

In [ ]:
clicks_df.printSchema()

root
 |-- productSKU: string (nullable = true)
 |-- totalClicks: integer (nullable = true)
 |-- totalImpressions: integer (nullable = true)
 |-- day: date (nullable = true)
 |-- storeId: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- model: string (nullable = true)
 |-- convertion_perc: double (nullable = true)



In [19]:
clicks_df.select(f.min('day'), f.max('day'), f.datediff(f.max('day'), f.min('day')).alias('range_days')).show()

+----------+----------+----------+
|  min(day)|  max(day)|range_days|
+----------+----------+----------+
|2023-06-01|2023-06-30|        29|
+----------+----------+----------+



In [20]:
clicks_df.select('day').distinct().count()

30

In [20]:
ads_by_days = clicks_df.groupBy('day', 'storeId').count().orderBy('day').toPandas()

In [21]:
fig = px.line(ads_by_days, x="day", y="count", color ='storeId',  title='Quantidade de anúncios por dia')
fig.update_layout(width = 800, height = 500, paper_bgcolor = 'white', plot_bgcolor = 'white')

fig.show()

In [22]:
clicks_df.where('day < "2023-06-29"')\
            .withColumn('day_of_week', f.dayofweek(f.col('day')))\
            .groupBy('day_of_week').agg(f.countDistinct('day'))\
            .orderBy('day_of_week').show()

+-----------+----------+
|day_of_week|count(day)|
+-----------+----------+
|          1|         4|
|          2|         4|
|          3|         4|
|          4|         4|
|          5|         4|
|          6|         4|
|          7|         4|
+-----------+----------+



In [23]:
ads_by_weekday = clicks_df.where('day < "2023-06-29"')\
                            .withColumn("day_of_week", f.dayofweek(f.col("day")))\
                            .groupBy('day_of_week', 'storeId').count()\
                            .orderBy('day_of_week', 'storeId').toPandas()

In [24]:
weekday_mapping = {
    1: 'Domingo',
    2: 'Segunda-feira',
    3: 'Terça-feira',
    4: 'Quarta-feira',
    5: 'Quinta-feira',
    6: 'Sexta-feira',
    7: 'Sábado'
}
ads_by_weekday['day_of_week'] = ads_by_weekday['day_of_week'].replace(weekday_mapping)

In [25]:
fig = px.bar(ads_by_weekday, x="day_of_week", y="count",
             color='storeId', barmode='group',
             title = 'Quantidade de anúncios por dia da semana', 
             labels = {'day_of_week': 'Dia da semana', 'count': 'Quantidade de anúncios', 'storeId': 'Loja'})
fig.update_layout(width = 800, height = 500, paper_bgcolor = 'white', plot_bgcolor = 'white')

fig.show()

In [26]:
avg_convertion_by_days = clicks_df.groupBy('day', 'storeId').agg(f.avg('convertion_perc').alias('average_convertion'))\
                            .orderBy('day').toPandas()

In [27]:
fig = px.line(avg_convertion_by_days, x="day", y="average_convertion", color ='storeId',  title='Taxa de conversão média')
fig.update_layout(width = 800, height = 500, paper_bgcolor = 'white', plot_bgcolor = 'white')

fig.show()

In [28]:
avg_convertion_by_weekday = clicks_df.withColumn('day_of_week', f.dayofweek(f.col('day')))\
                            .groupBy('day_of_week', 'storeId').agg(f.avg('convertion_perc').alias('average_convertion'))\
                            .orderBy('day_of_week', 'storeId').toPandas()

In [29]:
avg_convertion_by_weekday['day_of_week'] = avg_convertion_by_weekday['day_of_week'].replace(weekday_mapping)

In [30]:
fig = px.bar(avg_convertion_by_weekday, x="day_of_week", y="average_convertion",
             color='storeId', barmode='group',
             title = 'Taxa de conversão por dia da semana', 
             labels = {'day_of_week': 'Dia da semana', 'average_convertion': 'Média de conversão', 'storeId': 'Loja'})
fig.update_layout(width = 800, height = 500, paper_bgcolor = 'white', plot_bgcolor = 'white')

fig.show()